In [1]:
import pint
ureg = pint.UnitRegistry()
# ureg.define('dollar = dimensionless')
# dollar = ureg.Quantity(1, 'dollar')
dollar = 1

In [2]:
# Status Quo (Diesel)
# source: https://www.thetruckersreport.com/infographics/cost-of-trucking/
# alternate source: https://www.freightwaves.com/news/understanding-total-operating-cost-per-mile#:~:text=Typically%2C%20gross%20fuel%20expense%20averages%20between%20%240.40%2D%240.55%20per%20mile.
per_mile_us_shipping_cost = 1.38 * dollar / (ureg.mile)
per_mile_fuel_cost = 0.37 * dollar / (ureg.mile)
per_mile_equipment_cost = 0.24 * dollar / (ureg.mile)
per_mile_driver_cost = 0.55 * dollar / (ureg.mile)
per_mile_maintenance_cost = 0.12 * dollar / (ureg.mile)
per_mile_insurance_cost = 0.05 * dollar / (ureg.mile)


# Status Quo (Electric)

# source: https://apps.dana.com/commercial-vehicles/tco/
# alt source: https://avt.inl.gov/sites/default/files/pdf/fsev/costs.pdf
# retail
per_mile_electricity_cost = 0.24 * dollar / (ureg.mile)
# PPA
# https://pv-magazine-usa.com/2020/02/04/utility-scale-solar-ppa-pricing-down-4-7-in-2019-with-13-6-gw-of-corporate-deals-signed/#:~:text=Utility%2Dscale%20solar%20power%20purchase,kWh%2C%20according%20to%20LevelTen%20Energy.
# + transmission costs
# https://spectrum.ieee.org/energywise/energy/policy/how-to-predict-a-utilitys-transmission-and-distribution-costs-count-customers-served
per_mile_electricity_cost = 0.17 * dollar / (ureg.mile)

# source: https://apps.dana.com/commercial-vehicles/tco/
per_mile_electric_maintenance_cost = 0.3 * per_mile_maintenance_cost

# based on Tesla semi
# source: https://electrek.co/2018/05/02/tesla-semi-production-version-range-increase-elon-musk/
# alt source: https://lynceans.org/wp-content/uploads/2020/04/Tesla-Semi-converted.pdf
# Important note - this says a Tesla semi will have ~20% lower operating costs,
# Tesla range optimism is 27% https://www.caranddriver.com/features/a33824052/adjustment-factor-tesla-uses-for-big-epa-range-numbers/
per_mile_electric_energy_use = 947 * ureg.kWh / (600 * ureg.mile * 0.77)

In [3]:
# Proposal for first route, since LCVs are legal on this route and it has high volume
# source: https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/nhsavglhft2015.htm
pgh_to_chi =500 * ureg.miles

In [13]:
# Best battery for application: Lithium Iron Phosphate
# source: https://batteryuniversity.com/learn/article/types_of_lithium_ion
# High cycles
# Cheap
# Good low temp performance
# Safe (not nearly as suscpetible to thermal runaway as compared to Cobalt-based cathodes in Tesla Semi)

# source: https://www.alibaba.com/product-detail/Graphene-lithium-iron-phosphate-battery-3_1600131559793.html
# battery_price_per_kwh = (63 * dollar / (272 * ureg.amp * ureg.hour * 3.2 * ureg.volt)).to('1/kWh')
# this price is more realistic
battery_price_per_kwh = (80 * dollar / ureg.kilowatt_hour).to('1/kWh')
battery_lifespan = 3000 * pgh_to_chi
depletion_factor = 0.8 # how much capacity it has after full cycles
lfp_density = (220 * ureg.amp * ureg.hour * 3.2 * ureg.volt) / (5.4 * ureg.kg)
print((lfp_density / (ureg.kWh)).to('1/kg')*(ureg.kWh))
lfp_vol_density = (272 * ureg.amp * ureg.hour * 3.2 * ureg.volt) / (205 * ureg.mm * 72 * ureg.mm * 175 * ureg.mm)

# # NCA https://batteryuniversity.com/learn/article/types_of_lithium_ion
# # https://www.alibaba.com/product-detail/Nca-Battery-21700-NCA-3-65V_1600113267932.html?spm=a2700.galleryofferlist.normal_offer.d_title.6a1624eddNa9ff&s=p
# battery_price_per_kwh = (2.70 * dollar / (4.5 * ureg.amp * ureg.hour * 3.65 * ureg.volt)).to('1/kWh')
# print(battery_price_per_kwh)
# battery_lifespan = 500 * pgh_to_chi
# depletion_factor = 0.8 # how much capacity it has after full cycles
# lfp_density = (4.5 * ureg.amp * ureg.hour * 3.65 * ureg.volt) / (0.072 * ureg.kg)
# lfp_vol_density = (4.5 * ureg.amp * ureg.hour * 3.65 * ureg.volt) / (100 * ureg.mm * 50 * ureg.mm * 50 * ureg.mm)

0.13037037037037036 kilowatt_hour / kilogram


In [19]:
# Parameters that can vary. Fiddle with these to test different operating conditions\
# If you want to test different configurations, these variables can be changed

# How much work (proportionally) the electric motors will do (1 means 0% diesel, 0 means 100% diesel)
proportion_to_electric = 0.7

# How much of the cost savings we claim as revenue, the rest passed on to the rest of the market
# (carriers, owner/operators, etc)
prop_take = 0.6

# How much more the tolls will cost by adding a 1-axle vehicle to the train
# source: https://www.ohioturnpike.org/docs/default-source/annual-report-files/cafr-2019-final.pdf?sfvrsn=d361ebc4_2
# when >3 trailers, ie classified as a road train (A type, ie fifth wheel to fifth wheel, 53' trailers)
# per_mile_toll_increase = 0.07 * dollar / ureg.mile
# when no increase in trailers, ie classified as a road train (B type, ie tow hitch to fifth wheel, twin 28' trailer LTL market)
# we hope to generally negotiate additional weight fees to the state/tollway to 75% their stated price. Plus, most roads don't have tolls
per_mile_toll_increase = 0.07 * dollar / ureg.mile * 0.75

# avg speed when on road
sixth_wheel_speed = 55 * ureg.mph

 # proportion of day on road
# 30 min charge per trip, 60 min waiting in yard for next trip
# https://en.wikipedia.org/wiki/Lithium_iron_phosphate_battery
sixth_wheel_utilization = pgh_to_chi / sixth_wheel_speed / (pgh_to_chi / sixth_wheel_speed + 1.5 * ureg.hour)
print("utilization", sixth_wheel_utilization)

# cost of components (as a proportion of battery cost), roughly based on Tesla
additional_component_cost = 15000

utilization 0.8583690987124464 dimensionless


In [20]:
# iteratively get capacity, since more battery weight means need for more capacity
added_weight = 10000 * ureg.pound
percentage_fuel_per_pound = (0.5 / 100) / (1000 * ureg.pound)
resultant_weight = added_weight*0.9
# proportional weight of components (as a proportion of battery cost)
additional_component_prop_weight = 0.25

# https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/docs/10factsfigures/table3_3.htm
# https://www.freightpros.com/blog/less-than-truckload/
# https://www.terrybryant.com/how-much-does-semi-truck-weigh#:~:text=Semi%2Dtractors%20weigh%20up%20to,haul%20up%20to%2034%2C000%20pounds.
base_truck_weight = 70000 * ureg.pound
percent_under_base_weight = 0.8

# How weight impacts fuel https://www.internationaltrucks.com/en/blog/fuel-economy-weight
# 0.5% increase per 1000lbs

while abs(added_weight - resultant_weight) > 1 * ureg.pound:
    added_weight = resultant_weight
    battery_capacity = proportion_to_electric * per_mile_electric_energy_use * pgh_to_chi * (1 + percentage_fuel_per_pound * added_weight) / depletion_factor
    battery_price = battery_price_per_kwh * battery_capacity

    lfp_battery_weight = battery_capacity / lfp_density

    resultant_weight = lfp_battery_weight*(1+additional_component_prop_weight)
    
added_weight = resultant_weight

added_weight_factor = (1 + percentage_fuel_per_pound * added_weight)

single_battery_price = 27 * dollar
num_batteries = battery_price / single_battery_price
    
print("total truck weight (must be <80k lbs)", (base_truck_weight + added_weight).to('lb'))
print("6w weight (must be <80k lbs)", added_weight.to('lb'))
print("battery compartment length", ((battery_capacity / lfp_vol_density) / (110 * ureg.inch * 100 * ureg.inch)).to('feet'))
print("battery capacity", battery_capacity)

total truck weight (must be <80k lbs) 90940.9266035365 pound
6w weight (must be <80k lbs) 20940.9266035365 pound
battery compartment length 1.3591298920808348 foot
battery capacity 990.6734441153033 kilowatt_hour


In [21]:
# operating costs, averaged as a funtion of the proportion converted to electricity
sixth_wheel_per_mile_fuel_cost = (per_mile_fuel_cost * (1-proportion_to_electric) + \
                                    per_mile_electricity_cost * (proportion_to_electric)) * added_weight_factor
sixth_wheel_per_mile_maintenance_cost = (per_mile_maintenance_cost * (1-proportion_to_electric) + \
                                    per_mile_electric_maintenance_cost * (proportion_to_electric)) * added_weight_factor
# Assume this is slightly lower because of ADAS features like ABS, ACC, anti-skid
sixth_wheel_per_mile_insurance_cost = 0.8 * per_mile_insurance_cost

# PRICING
delta = -per_mile_toll_increase + (per_mile_fuel_cost + per_mile_maintenance_cost + per_mile_insurance_cost) - \
        (sixth_wheel_per_mile_fuel_cost + sixth_wheel_per_mile_maintenance_cost + sixth_wheel_per_mile_insurance_cost)
sixth_wheel_per_mile_rental_cost = prop_take * delta

In [22]:

 # proportion of day on road# Total cost savings

sixth_wheel_per_mile_cost_no_cut = per_mile_us_shipping_cost - delta

overall_savings_no_cut = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost_no_cut) / per_mile_us_shipping_cost * 100
driver_savings_no_cut = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost_no_cut) / per_mile_driver_cost * 100
print("Overall percentage savings (WITHOUT OUR CUT) %", overall_savings_no_cut)
print("Potential driver wage increase (if given only to driver, WITHOUT OUR CUT) %", driver_savings_no_cut)

sixth_wheel_per_mile_cost = per_mile_us_shipping_cost - delta + \
                            + sixth_wheel_per_mile_rental_cost

overall_savings = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost) / per_mile_us_shipping_cost * 100
driver_savings = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost) / per_mile_driver_cost * 100

print("Overall percentage savings (WITH OUR CUT) %", overall_savings)
print("Potential driver wage increase (if given only to driver, WITH OUR CUT) %", driver_savings)

Overall percentage savings (WITHOUT OUR CUT) % 9.11666745400369 dimensionless
Potential driver wage increase (if given only to driver, WITHOUT OUR CUT) % 22.874547430045617 dimensionless
Overall percentage savings (WITH OUR CUT) % 3.646666981601479 dimensionless
Potential driver wage increase (if given only to driver, WITH OUR CUT) % 9.149818972018254 dimensionless


In [25]:
#  Capital analysis

sixth_wheel_capital_expense = battery_price + additional_component_cost
sixth_wheel_miles_per_year = (sixth_wheel_utilization * sixth_wheel_speed * ureg.year).to('mile')
sixth_wheel_annual_revenue = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost).to('dimensionless')

print("Cost of each sixth wheel", sixth_wheel_capital_expense)
# determine margins
# yard worker wages, etc. THIS IS A ROUGH ESTIMATE. Will serve as a target and someting to drive down
# https://missionfinancialservices.net/what-are-your-options-to-cover-the-cost-of-semi-truck-repair/#:~:text=Most%20importantly%2C%20year%20truck%20maintenance,tires%20on%20an%2018%20wheeler.
other_annual_operating_cost = 0.3 *15000 + 4000 *2/16
# assume 3 yr loan @ 4.2% https://www.commercialloandirect.com/commercial-rates.php
loan_years = 3
rate = 3.75/1200;
months = loan_years*12;
debt_annual = 12*sixth_wheel_capital_expense * rate * (1+rate)**months / ((1+rate)**months - 1)
print("sixth wheel debt service annually", debt_annual)
print("sixth wheel debt liability total", debt_annual*loan_years)
print("sixth wheel annual revenue", sixth_wheel_annual_revenue)

sixth_wheel_margins = (sixth_wheel_annual_revenue - (other_annual_operating_cost + debt_annual)) / sixth_wheel_annual_revenue

print("operating margins", sixth_wheel_margins)

lifetime_profit = sixth_wheel_per_mile_rental_cost * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins
print("lifetime profit", lifetime_profit)
lifetime_value = sixth_wheel_per_mile_rental_cost * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year)
print("lifetime value", lifetime_value)
print("years until 6w battery failure", battery_lifespan/sixth_wheel_miles_per_year)
left_on_loan = debt_annual*loan_years - battery_lifespan/sixth_wheel_miles_per_year * (debt_annual)
print("payments left on loan", left_on_loan)
print("profit after loan payoff", lifetime_profit - left_on_loan)
print("annualized profit per 6W", (lifetime_profit - left_on_loan) / loan_years)
print("capital accrued", (lifetime_value - other_annual_operating_cost*loan_years + additional_component_cost))
print("6W rate of return", 100*(((lifetime_value - other_annual_operating_cost*loan_years + additional_component_cost) / sixth_wheel_capital_expense)**(1/loan_years) - 1))
print("sixth wheel payback period", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue - other_annual_operating_cost))
print("sixth wheel oeprating costs", (sixth_wheel_annual_revenue * (1-sixth_wheel_margins)))
print("sixth wheel (rough) capital efficiency (%)", (sixth_wheel_annual_revenue - other_annual_operating_cost) / (sixth_wheel_capital_expense) * 100)

# source: https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/nhsavglhft2015.htm
pgh_to_chi_daily_trucks = 25000 
pgh_to_chi_tam_no_yard = 365 * pgh_to_chi_daily_trucks * pgh_to_chi * sixth_wheel_per_mile_rental_cost
print("PGH to CHI no yard TAM (not including split between our take and that given to market) ($)", percent_under_base_weight * pgh_to_chi_tam_no_yard)

# source: https://www.commerceexpressinc.com/2020/01/09/5-quick-facts-about-semi-trucks/#:~:text=Engines%20can't%20last%20forever,%2Dtruck%2015%2D16%20years.
# source: https://hdstruckdrivinginstitute.com/semi-trucks-numbers/#:~:text=There%20are%20nearly%202%20million,tractors%20and%20tractor%20trailers%2C%20respectively.
annual_semi_miles = 140e9*ureg.miles * 0.42
print("percent under base weight", percent_under_base_weight)
print("Total US trucking no yard TAM (not including split between our take and that given to market) ($)", percent_under_base_weight * annual_semi_miles * sixth_wheel_per_mile_rental_cost)

Cost of each sixth wheel 94253.87552922426 dimensionless
sixth wheel debt service annually 33267.36121356697 dimensionless
sixth wheel debt liability total 99802.0836407009 dimensionless
sixth wheel annual revenue 31239.54362495967 dimensionless
operating margins -0.22496543717086293 dimensionless
lifetime profit -25472.613685294942 dimensionless
lifetime value 113229.00977872571 dimensionless
years until 6w battery failure 3.6245410988737476 dimensionless
payments left on loan -20776.83432895101 dimensionless
profit after loan payoff -4695.779356343934 dimensionless
annualized profit per 6W -1565.259785447978 dimensionless
capital accrued 113229.00977872571 dimensionless
6W rate of return 6.304789271264433 dimensionless
sixth wheel payback period 3.5920546819102355 year
sixth wheel oeprating costs 38267.36121356697 dimensionless
sixth wheel (rough) capital efficiency (%) 27.839219849186854 dimensionless
PGH to CHI no yard TAM (not including split between our take and that given to mar

In [10]:
#  Capital analysis, other revenue streams
#  Yard operations (using sixth wheel as a yard jockey)
#  source: baton.io
detention_time_per_trip = 2 * ureg.hours
percent_time_improvement = 0.7
trips_per_year = pgh_to_chi_daily_trucks * 365 * percent_time_improvement
hourly_wages = 24.48 * dollar / ureg.hour # source: https://www.bls.gov/ooh/transportation-and-material-moving/material-moving-machine-operators.htm
pgh_to_chi_tam_yard = detention_time_per_trip * hourly_wages * trips_per_year
# NOTE: making the sixth wheels autonomous yard jockeys will also improve margins on this market
print("PGH to CHI yard TAM", percent_under_base_weight * pgh_to_chi_tam_yard)

average_trip_length = 500 * ureg.miles
print("Total US trucking yard TAM", percent_under_base_weight * detention_time_per_trip * hourly_wages * (annual_semi_miles / average_trip_length))

sixth_wheel_per_mile_rental_cost_with_yard = sixth_wheel_per_mile_rental_cost + detention_time_per_trip * hourly_wages / average_trip_length
sixth_wheel_annual_revenue_with_yard = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost_with_yard).to('dimensionless')

print("lifetime value w/ yard", sixth_wheel_per_mile_rental_cost_with_yard * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins)
print("sixth wheel payback period w/ yard", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue_with_yard - other_annual_operating_cost))
print("sixth wheel (rough) capital efficiency  w/ yard (%)", (sixth_wheel_annual_revenue_with_yard - other_annual_operating_cost)/ (sixth_wheel_capital_expense ) * 100)


PGH to CHI yard TAM 250185600.0 dimensionless
Total US trucking yard TAM 4606156800.000001 dimensionless
lifetime value w/ yard -283558.5025544291 dimensionless
sixth wheel payback period w/ yard 1.375019555085373 year
sixth wheel (rough) capital efficiency  w/ yard (%) 72.72623842341729 dimensionless


In [11]:
#  Capital analysis, other revenue streams
#  Autonomy (when full autonomy is achieved)
sixth_wheel_per_mile_driver_cost_autonomy = 0.15 * dollar / (ureg.mile) # software license fee to AV company

delta_autonomy = -per_mile_toll_increase + (per_mile_fuel_cost + per_mile_maintenance_cost + per_mile_insurance_cost + per_mile_driver_cost) - \
        (sixth_wheel_per_mile_fuel_cost + sixth_wheel_per_mile_maintenance_cost + sixth_wheel_per_mile_insurance_cost + sixth_wheel_per_mile_driver_cost_autonomy)
sixth_wheel_per_mile_rental_cost_autonomy = prop_take * delta_autonomy

sixth_wheel_annual_revenue_autonomy = (365 * pgh_to_chi_daily_trucks * pgh_to_chi * sixth_wheel_per_mile_rental_cost_autonomy).to('dimensionless')
print("PGH to CHI annual revenue, autonomy", percent_under_base_weight * sixth_wheel_annual_revenue_autonomy)

print("Total US trucking annual revenue, autonomy", percent_under_base_weight * annual_semi_miles * sixth_wheel_per_mile_rental_cost_autonomy)

sixth_wheel_per_mile_rental_cost_with_yard_autonomy = sixth_wheel_per_mile_rental_cost_autonomy + detention_time_per_trip * hourly_wages / average_trip_length
sixth_wheel_annual_revenue_with_yard_autonomy = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost_with_yard_autonomy).to('dimensionless')

print("lifetime value  w/ yard and AV", sixth_wheel_per_mile_rental_cost_with_yard_autonomy * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins)
print("sixth wheel payback period  w/ yard and AV", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue_with_yard_autonomy - other_annual_operating_cost))
print("sixth wheel (rough) capital efficiency  w/ yard and AV (%)", (sixth_wheel_annual_revenue_with_yard_autonomy - other_annual_operating_cost)/ (sixth_wheel_capital_expense ) * 100)


PGH to CHI annual revenue, autonomy 865468493.833982 dimensionless
Total US trucking annual revenue, autonomy 11153873410.945343 dimensionless
lifetime value  w/ yard and AV -703537.36024866 dimensionless
sixth wheel payback period  w/ yard and AV 0.5234899613411931 year
sixth wheel (rough) capital efficiency  w/ yard and AV (%) 191.02563064208098 dimensionless


In [12]:
# cost per ton of CO2 reduced
mpg = 7 * ureg.mile / ureg.gallon
gpm_status_quo = 1/ mpg
gpm_sixth_wheel = (1-proportion_to_electric) * added_weight_factor / mpg
# https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf
diesel_emissions = 10.21 * ureg.kg / ureg.gallon
print("tons saved per mile", ((gpm_status_quo - gpm_sixth_wheel) * diesel_emissions).to('ton/mile'))
print("delta", delta)
print("lifetime tons saved", ((gpm_status_quo - gpm_sixth_wheel) * diesel_emissions * battery_lifespan).to('ton'))
print("cost per ton, lifetime", (sixth_wheel_capital_expense / ((gpm_status_quo - gpm_sixth_wheel) * diesel_emissions * battery_lifespan)).to('1/ton'))
print("savings per ton", (delta *sixth_wheel_margins/((gpm_status_quo - gpm_sixth_wheel) * diesel_emissions)).to('1/ton'))

tons saved per mile 0.0007455231572784508 ton / mile
delta 0.07422931168985303 / mile
lifetime tons saved 1677.4271038765144 ton
cost per ton, lifetime 41.71017929118081 / ton
savings per ton -92.92428105784457 / ton
